# Analyzing OpenStreetMap data

In this document I would like to show you a MongoDB queries I used to analyze OpenStreetMap data.

## Import pymogno and acessing database

In [8]:
import pymongo, pprint

client = pymongo.MongoClient('mongodb://localhost:27017/')

db = client.openstreetmap

## Data overview

In [18]:
print('Total elements in database: {}'.format(db.elements.count()))

Total elements in database: 52406


In [41]:
def number_of_elements(node_type):
    count_of_type = db.elements.aggregate([
        {'$match': {'type': node_type}},
        {'$group': {'_id': None, 'count': {'$sum': 1}}}
    ])
    
    return list(count_of_type)[0]['count']

In [43]:
print('Total nodes: {}'.format(number_of_elements('node')))

Total nodes: 46957


In [44]:
print('Total ways: {}'.format(number_of_elements('way')))

Total ways: 5362


In [45]:
print('Total relations: {}'.format(number_of_elements('relation')))

Total relations: 87
